In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# データの例を作成
data = {
    'date': ['31-Mar-19', '31-Mar-19', '31-Mar-19'],
    'time': ['12:36:52', '12:36:53', '12:36:54'],
    'fridge_temperature': [13.1, 8.65, 2],
    'temp_condition': ['high', 'high', 'low'],
    'label': [0, 0, 1],
    'type': ['normal', 'normal', 'ddos']
}

df = pd.DataFrame(data)

# temp_conditionのワンホットエンコーディング
encoder = OneHotEncoder(sparse=False)
encoded_conditions = encoder.fit_transform(df[['temp_condition']])
condition_df = pd.DataFrame(encoded_conditions, columns=encoder.get_feature_names_out())

# データフレームを統合
df = df.join(condition_df)
df.drop(['date', 'time', 'temp_condition', 'type'], axis=1, inplace=True)

# 特徴量とラベルを分割
X = df.drop('label', axis=1)
y = df['label']

# データをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 特徴量のスケーリング
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


/home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [2]:
X_test_scaled

array([[ 2.33834586,  1.        , -1.        ]])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# モデルの構築
model = Sequential([
    LSTM(50, input_shape=(X_train_scaled.shape[1], 1)),  # 入力形状に注意
    Dense(1, activation='sigmoid')
])

# モデルのコンパイル
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# LSTMに合うようにデータの形状を変更
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# モデルのトレーニング
history = model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))


In [ ]:
# モデルの評価
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")
